In [54]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from torch import nn, optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

df_train = pd.read_csv('../../house_prise/input/train.csv')
df_test = pd.read_csv('../../house_prise/input/test.csv')
df = df_train.append(df_test, ignore_index=True, sort=False)
df = pd.get_dummies(df, dummy_na=True, drop_first=True)
df.drop("Id", axis=1, inplace=True)
df.isnull().values.any()

df.fillna(df.median(), inplace=True)
sale_price = df['SalePrice']
df.isnull().values.any()

display(df.head())

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleType_nan,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan
0,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,...,0,0,1,0,0,0,0,1,0,0
1,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,...,0,0,1,0,0,0,0,1,0,0
2,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,...,0,0,1,0,0,0,0,1,0,0
3,70,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,...,0,0,1,0,0,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,...,0,0,1,0,0,0,0,1,0,0


In [55]:
columns = df.columns
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns = columns)
df['SalePrice'] = sale_price
df.head()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleType_nan,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan
0,0.235294,0.150685,0.033420,0.666667,0.500,0.949275,0.883333,0.12250,0.125089,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.000000,0.202055,0.038795,0.555556,0.875,0.753623,0.433333,0.00000,0.173281,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.235294,0.160959,0.046507,0.666667,0.500,0.934783,0.866667,0.10125,0.086109,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.294118,0.133562,0.038561,0.666667,0.500,0.311594,0.333333,0.00000,0.038271,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.235294,0.215753,0.060576,0.777778,0.500,0.927536,0.833333,0.21875,0.116052,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [56]:
len(df_train)

train = df.iloc[:len(df_train)]
test = df.iloc[len(df_train):]
test.drop('SalePrice', axis=1, inplace=True)
print(train.shape)
print(test.shape)

(1460, 289)
(1459, 288)


/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [57]:
x_train, x_val, y_train, y_val = train_test_split(train.drop("SalePrice", axis=1), train['SalePrice'], test_size=0.2, random_state=1208)
print(x_train.shape)
print(x_val.shape)

(1168, 288)
(292, 288)


In [58]:
x_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleType_nan,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan
105,0.235294,0.184932,0.037743,0.777778,0.500,0.949275,0.900000,0.4800,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
968,0.176471,0.099315,0.021618,0.222222,0.625,0.275362,0.000000,0.0000,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
507,0.000000,0.184932,0.030671,0.555556,0.500,0.992754,0.983333,0.0000,0.004784,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
272,0.235294,0.243151,0.048910,0.777778,0.750,0.920290,0.950000,0.2175,0.092842,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
803,0.235294,0.294521,0.058852,0.888889,0.500,0.985507,0.983333,0.2650,0.000000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [59]:
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(288, 144)
        self.fc2 = nn.Linear(144, 72)
        self.fc3 = nn.Linear(72, 18)
        self.fc4 = nn.Linear(18, 1)
        
        # self.dropout = nn.Dropout(p=0.1)
        
    def forward(self, x):
        
#         x = self.dropout(F.relu(self.fc1(x)))
#         x = self.dropout(F.relu(self.fc2(x)))
#         x = self.dropout(F.relu(self.fc3(x)))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        
        return x

In [60]:
batch_size = 50
train_batch = np.array_split(x_train, batch_size)
label_batch = np.array_split(y_train, batch_size)
len(train_batch)

50

In [61]:
for i in range(len(train_batch)):
    train_batch[i] = torch.from_numpy(train_batch[i].values).float()
for i in range(len(label_batch)):
    label_batch[i] = torch.from_numpy(label_batch[i].values).float().view(-1, 1) # (24)を(24,1)にする。

x_val = torch.from_numpy(x_val.values).float()
y_val = torch.from_numpy(y_val.values).float().view(-1, 1) # 1次元Tensorじゃだめなんだ

In [62]:
model = Regressor()
ps = model(train_batch[0])
ps.shape

torch.Size([24, 1])

In [ ]:
model = Regressor()
# Loss Function
criterion = nn.MSELoss()
# Optimize Function
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 300

train_losses, test_losses = [], []

for ei in range(epochs):
    model.train()
    train_loss = 0
    
    for i in range(len(train_batch)):
        # 勾配情報の初期化
        optimizer.zero_grad()
        # バッチ毎に学習を行う
        output = model(train_batch[i])
        # バッチの損失
        loss = torch.sqrt(criterion(torch.log(output), torch.log(label_batch[i])))
        # 逆伝播
        loss.backward()
        # 勾配情報の更新
        optimizer.step()
        
        train_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
        
        with torch.no_grad():
            model.eval()
            predictions = model(x_val)
            test_loss += torch.sqrt(criterion(torch.log(predictions), torch.log(y_val)))
            
        train_losses.append(train_loss/len(train_batch))
        test

In [ ]:
import torch
import torch.nn as nn

# 各シーケンス長
seq_len_list = [8, 6, 10]

input_size = 2
hidden_size = 4

# Input Data Sample
input_seq = []
for seq_len in seq_len_list:
    iseq = torch.randn(seq_len, input_size)
    print('input sequence: ', iseq.shape)
    input_seq.append(iseq)
    
# Network Definition
net = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)

# Forward
output_seq = []
for iseq in input_seq:
    iseq = iseq.unsqueeze(0) # 先頭にバッチサイズの次元1を加える
    print("input sequence add batch_size index 0", iseq.shape)
    oseq, _ = net(iseq)
    output_seq.append(oseq.squeeze(0)) # squeezeで先頭のバッチサイズの次元1を削除してリストに追加
    
# Check Output Data    
for oseq in output_seq:
    print("output sequence:", oseq.shape)
    
    
#========================================================================

batch_size = 20
seq_len = 10

input_size = 2
hidden_size = 4

# Input Data Sample
input_seq = torch.randn(batch_size, seq_len, input_size)
print("Input Sequence:", input_seq.shape)

# Network Definition
net = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)

# 順伝搬
output_seq, _ = net(input_seq)

# 出力データの確認
print("output sequence:", output_seq.shape)

#========================================================================

batch_size = 20
seq_len = 10

input_size = 2
hidden_size = 4

# Input Data Sample
input_seq = torch.randn(batch_size, seq_len, input_size)
print("input sequence:", input_seq.size())

# Network Definition
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # LSTM Block A
        self.block_a = nn.LSTMCell(input_size, hidden_size)
        # LSTM Block B
        self.block_b = nn.LSTMCell(hidden_size, hidden_size)
        
    def forward(self, x, hx_a0, cx_a0, hx_b0, cx_b0):
        '''
        :param x: 現ステップの入力シーケンスの状態
        :param hx_a0: 前ステップのLSTM Block Aの隠れ層の状態
        :param cx_a0: 前ステップのLSTM Block Aのセル状態
        :param hx_b0: 前ステップのLSTM Block Bの隠れ層の状態
        :param cx_b0: 前ステップのLSTM Block Bのセル状態
        :return: 現ステップのLSTMブロックAの隠れ層の状態とセル状態、現ステップのLSTMブロックBの隠れ層の状態とセル状態
        '''
        
        hx_a1, cx_a1 = self.block_a(x, (hx_a0, cx_a0)) # 現ステップのLSTMブロックAの中間層の状態とセル状態
        hx_b1, cx_b1 = self.block_b(hx_a1, (hx_b0, cx_b0)) # 現ステップのLSTMブロックBの中間層の状態とセル状態
        return hx_a1, cx_a1, hx_b1, cx_b1
        
net = Net()

# 中間層の初期化
hx_a = torch.randn(batch_size, hidden_size)
cx_a = torch.randn(batch_size, hidden_size)
hx_b = torch.randn(batch_size, hidden_size)
cx_b = torch.randn(batch_size, hidden_size)

# 順伝搬
output_seq = []
for i in range(seq_len):
    hx_a, cx_a, hx_b, cx_b = net(input_seq[:, i, :], hx_a, cx_a, hx_b, cx_b)
    output_seq.append(hx_b)

# リストからTensorに変換
output_seq = torch.stack(output_seq, dim=1)

# Check Output Data
print("output sequence:", output_seq.shape)

#========================================================================

seq_len_list = [8, 6, 10]

input_size = 2
hidden_size = 4

# Input Data Sample
input_seq = []
for seq_len in seq_len_list:
    iseq = torch.randn(seq_len, input_size)
    print("input sequence:", iseq.shape)
    input_seq.append(iseq)
    
# Network Definition
net = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)

# 順伝播
output_data = []

for iseq in input_seq:
    iseq = iseq.unsqueeze(0) # 先頭にバッチサイズの次元1を加える
    oseq, _ = net(iseq)
    
    # スライスでシーケンスの最終ステップのみ取り出し、squeezeで先頭のバッチサイズの次元1を除去してリストに追加
    output_data.append(oseq[:, -1, :].squeeze(0))
    
# check output data 
for odata in output_data:
    print("output data:", odata.shape)